# Fine Tuning 
## Training Set Prep

In [3]:
import os
import random
import json
import pathlib
import shutil
import tarfile
import boto3
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer, util

pd.set_option('display.max_columns', None)

In [2]:
os.getcwd()

'c:\\Users\\lwest\\Desktop\\magicml\\notebooks'

In [20]:
gpl_path = '../data/gpl/corpus.jsonl'

In [14]:
df = pd.read_json(gpl_path, lines=True)
print(df.shape)

df.query('text.notnull()').shape

(11573, 3)


(11444, 3)

In [25]:
cards_df = pd.read_csv('cards.csv')
print(cards_df.shape)

cards_df = cards_df\
  .query('text.notnull()')\
  .assign(title='')\
  .assign(id=lambda df: df.id.astype(str))\
  .rename(columns={'id':'_id'})\
  [['text','title','_id']]

cards_df.to_json('../data/gpl/corpus.jsonl', orient='records', lines=True)

print(cards_df.shape)
cards_df.head(2)

(11573, 95)
(11444, 3)


c:\Users\lwest\Anaconda3\envs\magicml\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (16,40) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,text,title,_id
0,Vigilance\nYou may look at the top card of you...,,4751
1,Deathtouch\nMystic Arcanum — At the beginning ...,,4752


In [22]:
supported_sets = list(pd.read_csv('supported_sets.txt', sep='\n', names=['sets']).sets)

supported_sets

['Capenna',
 'Kamigawa',
 'Innistrad',
 'Forgotten Realms',
 'Strixhaven',
 'Throne of Eldraine',
 'Theros Beyond Death',
 'Ikoria: Lair of Behemoths',
 'Core Set 2021',
 'Zendikar Rising',
 'Kaldheim',
 'Core Set 2020',
 'War of the Spark',
 'Ravnica Allegiance',
 'Guilds of Ravnica',
 'Ixalan',
 'Rivals of Ixalan',
 'Dominaria',
 'Core Set 2019',
 'Historic Anthology',
 'Historic Anthology 2',
 'Historic Anthology 3',
 'Jumpstart',
 'Amonkhet Remastered',
 'Kaladesh Remastered']

In [11]:
model_path = 'model.tar.gz'

In [12]:
os.getcwd()

'c:\\Users\\lwest\\Desktop\\magicml\\notebooks'

In [17]:
os.system('tar -xf {} -C {}'.format('../services/models/magicBERT/model.tar.gz', '.'))

0

In [4]:
with tarfile.open(model_path) as tar:
  tar.extractall(path='.')

In [4]:
model = SentenceTransformer('magicml-LM')

You try to use a model that was created with version 2.2.2, however, your version is 2.2.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





In [5]:
cards_df = pd.read_csv('cards.csv')[['name','text']]
print(cards_df.shape)
cards_df.head(3)

(11573, 2)


c:\Users\lwest\Anaconda3\envs\magicml\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (16,40) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,name,text
0,"Galea, Kindler of Hope",Vigilance\nYou may look at the top card of you...
1,"Prosper, Tome-Bound",Deathtouch\nMystic Arcanum — At the beginning ...
2,Sefris of the Hidden Ways,Whenever one or more creature cards are put in...


In [6]:
cards_df = pd.read_csv('cards.csv')\
  .reset_index(drop=True)\
  .fillna(value={'text': 'Blank'})

cards_txt = cards_df.text.to_list()
cards_name = [
  (name + '-' + str(id_val)).replace(' ','_').replace('//', 'II') for name, id_val in zip(cards_df.name, cards_df.id)
]

c:\Users\lwest\Anaconda3\envs\magicml\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (16,40) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [20]:
query_embed = model.encode(cards_txt[15])

In [19]:
embeddings.shape

(10, 768)

In [22]:
cos_sim = util.cos_sim(embeddings, query_embed).numpy()
cos_sim

array([[0.7855769 ],
       [0.73620105],
       [0.7894338 ],
       [0.721814  ],
       [0.7933928 ],
       [0.7824372 ],
       [0.7100925 ],
       [0.7695936 ],
       [0.7472745 ],
       [0.79078764]], dtype=float32)

In [20]:
#Encode all sentences
embeddings = model.encode(cards_txt[:1000])

#Compute cosine similarity between all pairs
cos_sim = util.cos_sim(embeddings, embeddings)

In [26]:
sim_df = pd.DataFrame(cos_sim.numpy(), columns=cards_name[0:1000], index=cards_name[0:1000])
sim_df.head(3)

,"Galea,_Kindler_of_Hope-4751","Prosper,_Tome-Bound-4752",Sefris_of_the_Hidden_Ways-4753,"Vrondiss,_Rage_of_Ancients-4754",Fey_Steed-4755,Holy_Avenger-4756,Immovable_Rod-4757,Mantle_of_the_Ancients-4758,Radiant_Solar-4759,Revivify-4760,Robe_of_Stars-4761,Thorough_Investigation-4762,Valiant_Endeavor-4763,Arcane_Endeavor-4764,Diviner's_Portent-4765,"Minn,_Wily_Illusionist-4766",Netherese_Puzzle-Ward-4767,Phantom_Steed-4768,Rod_of_Absorption-4769,Winged_Boots-4770,Bag_of_Devouring-4771,Danse_Macabre-4772,Death_Tyrant-4773,Grave_Endeavor-4774,Grim_Hireling-4775,Hellish_Rebuke-4776,"Lorcan,_Warlock_Collector-4777",Wand_of_Orcus-4778,Berserker's_Frenzy-4779,Chaos_Dragon-4780,Fiendlash-4781,Maddening_Hex-4782,Reckless_Endeavor-4783,Share_the_Spoils-4784,Vengeful_Ancestor-4785,Wild-Magic_Sorcerer-4786,Bag_of_Tricks-4787,Belt_of_Giant_Strength-4788,Druid_of_Purification-4789,Indomitable_Might-4790,Neverwinter_Hydra-4791,Song_of_Inspiration-4792,Wild_Endeavor-4793,Catti-brie_of_Mithral_Hall-4794,Dragonborn_Champion-4795,Extract_Brain-4796,Fevered_Suspicion-4797,Hurl_Through_Hell-4798,"Karazikar,_the_Eye_Tyrant-4799","Klauth,_Unrivaled_Ancient-4800",Klauth's_Will-4801,Midnight_Pathlighter-4802,Nihiloor-4803,Ride_the_Avalanche-4804,"Storvald,_Frost_Giant_Jarl-4805",Wulfgar_of_Icewind_Dale-4806,Bucknard's_Everfull_Purse-4807,Clay_Golem-4808,Component_Pouch-4809,Ebony_Fly-4810,Sword_of_Hours-4811,Underdark_Rift-4812,Angel_of_Finality-4813,Angelic_Gift-4814,Cataclysmic_Gearhulk-4815,Eternal_Dragon-4816,Gryff's_Boon-4817,Karmic_Guide-4818,Puresteel_Paladin-4819,Realm-Cloaked_Giant_II_Cast_Off-4820,Realm-Cloaked_Giant_II_Cast_Off-4821,Ronom_Unicorn-4822,"Sram,_Senior_Edificer-4823",Sun_Titan-4824,Sunblast_Angel-4825,Swords_to_Plowshares-4826,Valorous_Stance-4827,Wall_of_Omens-4828,Winds_of_Rath-4829,Brainstorm-4830,Champion_of_Wits-4831,Curator_of_Mysteries-4832,Curse_of_Verbosity-4833,Eel_Umbra-4834,Forbidden_Alchemy-4835,Imprisoned_in_the_Moon-4836,Merfolk_Looter-4837,Mulldrifter-4838,Murder_of_Crows-4839,Phantasmal_Image-4840,Prognostic_Sphinx-4841,Propaganda-4842,Psychic_Impetus-4843,Riverwise_Augur-4844,Serum_Visions-4845,Chittering_Witch-4846,Consuming_Vapors-4847,Dead_Man's_Chest-4848,Doomed_Necromancer-4849,Fiend_of_the_Shadows-4850,"Gonti,_Lord_of_Luxury-4851",Hex-4852,Marionette_Master-4853,Necromantic_Selection-4854,Ogre_Slumlord-4855,Phthisis-4856,Piper_of_the_Swarm-4857,Plaguecrafter-4858,Pontiff_of_Blight-4859,Reassembling_Skeleton-4860,Shriekmaw-4861,Unburial_Rites-4862,Victimize-4863,Anger-4864,Apex_of_Power-4865,Bogardan_Hellkite-4866,Chain_Reaction-4867,Chaos_Warp-4868,Commune_with_Lava-4869,Dark-Dweller_Oracle-4870,Demanding_Dragon-4871,Dire_Fleet_Daredevil-4872,Disrupt_Decorum-4873,Dragonlord's_Servant-4874,Dragonmaster_Outcast-4875,Dream_Pillager-4876,"Etali,_Primal_Storm-4877",Gratuitous_Violence-4878,Hoard-Smelter_Dragon-4879,Ignite_the_Future-4880,Izzet_Chemister-4881,Light_Up_the_Stage-4882,Loyal_Apprentice-4883,Magmaquake-4884,Opportunistic_Dragon-4885,Outpost_Siege-4886,Rile-4887,Scourge_of_Valkas-4888,Shiny_Impetus-4889,Shivan_Hellkite-4890,Skyline_Despot-4891,Skyship_Stalker-4892,Spit_Flame-4893,Taurean_Mauler-4894,Tectonic_Giant-4895,Terror_of_Mount_Velus-4896,Throes_of_Chaos-4897,Thunderbreak_Regent-4898,Vandalblast-4899,Warstorm_Surge-4900,Abundant_Growth-4901,Acidic_Slime-4902,Beast_Within-4903,Chameleon_Colossus-4904,Colossal_Majesty-4905,Cultivate-4906,Decree_of_Savagery-4907,Explore-4908,Fertile_Ground-4909,Garruk's_Uprising-4910,Greater_Good-4911,Heroic_Intervention-4912,Kenrith's_Transformation-4913,Kindred_Summons-4914,Nature's_Lore-4915,Paradise_Druid-4916,Rampant_Growth-4917,Rancor-4918,Return_of_the_Wildspeaker-4919,Return_to_Nature-4920,Rishkar's_Expertise-4921,Shamanic_Revelation-4922,Utopia_Sprawl-4923,Verdant_Embrace-4924,Wild_Growth-4925,Ashen_Rider-4926,"Atarka,_World_Render-4927",Baleful_Strix-4928,Bant_Charm-4929,Bedevil-4930,Behemoth_Sledge-4931,Bituminous_Blast-4932,Cloudblazer-4933,Cold-Eyed

In [42]:
sim_df\
  [['Puresteel_Paladin-4819']]\
  .sort_values(by='Puresteel_Paladin-4819', ascending=False)

,Puresteel_Paladin-4819
Puresteel_Paladin-4819,0.999999
Gretchen_Titchwillow-5422,0.846236
Gretchen_Titchwillow-5305,0.846236
"Galea,_Kindler_of_Hope-4751",0.838679
"Galea,_Kindler_of_Hope-5068",0.838679
...,...
Blink_Dog-5382,0.663843
Pride_Sovereign-6017,0.654759
Ancient_Crab-5853,0.645360
Dune_Beetle-5910,0.645360


In [29]:
name_txt_df = pd.DataFrame({'name': cards_name, 'text': cards_txt})
name_txt_df.head(2)

,name,text
0,"Galea,_Kindler_of_Hope-4751",Vigilance\nYou may look at the top card of you...
1,"Prosper,_Tome-Bound-4752",Deathtouch\nMystic Arcanum — At the beginning ...


In [43]:
names = ['Puresteel_Paladin-4819', 'Gretchen_Titchwillow-5422', 'Galea,_Kindler_of_Hope-4751']

In [44]:
name_txt_df.query('name == @names').text.to_list()

['Vigilance\nYou may look at the top card of your library any time.\nYou may cast Aura and Equipment spells from the top of your library. When you cast an Equipment spell this way, it gains "When this Equipment enters the battlefield, attach it to target creature you control."',
 'Whenever an Equipment enters the battlefield under your control, you may draw a card.\nMetalcraft — Equipment you control have equip {0} as long as you control three or more artifacts.',
 '{2}{G}{U}: Draw a card. You may put a land card from your hand onto the battlefield.']

In [8]:
#Add all pairs to a list with their cosine similarity score
all_sentence_combinations = []
for i in range(len(cos_sim)-1):
    for j in range(i+1, len(cos_sim)):
        all_sentence_combinations.append([cos_sim[i][j], i, j])

#Sort list by the highest cosine similarity score
all_sentence_combinations = sorted(all_sentence_combinations, key=lambda x: x[0], reverse=True)

print("Top-5 most similar pairs:")
for score, i, j in all_sentence_combinations[0:5]:
    print('Card 1: {}'.format(sentences[i]))
    print('Card 2: {}'.format(sentences[j]))
    print('Score: {}'.format(cos_sim[i][j]))

Top-5 most similar pairs:
Card 1: ({T}: Add {W}.)
Card 2: ({T}: Add {R}.)
Score: 0.9737533926963806
Card 1: ({T}: Add {W}.)
Card 2: ({T}: Add {U}.)
Score: 0.9661768078804016
Card 1: ({T}: Add {U}.)
Card 2: ({T}: Add {R}.)
Score: 0.9611330628395081
Card 1: Haste
Card 2: Flying
Score: 0.9022868871688843
Card 1: Trample
Fabricate 1 (When this creature enters the battlefield, put a +1/+1 counter on it or create a 1/1 colorless Servo artifact creature token.)
Card 2: Fabricate 3 (When this creature enters the battlefield, put three +1/+1 counters on it or create three 1/1 colorless Servo artifact creature tokens.)
Whenever an artifact you control is put into a graveyard from the battlefield, target opponent loses life equal to Marionette Master's power.
Score: 0.9002456665039062


## Tokenize and Pad

In [121]:
#from transformers import GPT2TokenizerFast

#tokenizer = GPT2TokenizerFast.from_pretrained('gpt2')
#tokenizer.pad_token = tokenizer.eos_token

In [11]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [35]:
tokenizer.model_max_length

512

In [96]:
txt_lens = [len(x.split(' ')) for x in train_texts]
max(txt_lens)

86

In [152]:
train_encodings = tokenizer(train_texts, padding='max_length', max_length=150)
#val_encodings = tokenizer(val_texts, truncation=True, padding=True)

In [37]:
tokenizer.save_pretrained('tokenizer')

('tokenizer\\tokenizer_config.json',
 'tokenizer\\special_tokens_map.json',
 'tokenizer\\vocab.txt',
 'tokenizer\\added_tokens.json',
 'tokenizer\\tokenizer.json')

In [39]:
from transformers import BertTokenizer
tokenizer2 = BertTokenizer.from_pretrained('tokenizer')

In [40]:
train_encodings = tokenizer2(train_texts, padding='max_length', max_length=150)

## Convert to TFDataset

In [153]:
import tensorflow as tf

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
))

#val_dataset = tf.data.Dataset.from_tensor_slices((
#    dict(val_encodings),
#    val_labels
#))

In [14]:
train_dataset.element_spec

({'input_ids': TensorSpec(shape=(150,), dtype=tf.int32, name=None),
  'token_type_ids': TensorSpec(shape=(150,), dtype=tf.int32, name=None),
  'attention_mask': TensorSpec(shape=(150,), dtype=tf.int32, name=None)},
 TensorSpec(shape=(), dtype=tf.int32, name=None))

In [154]:
input_size = train_dataset.element_spec[0]['input_ids'].shape[0]
input_size

150

In [146]:
train_dataset.element_spec[0].keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

## Train

In [155]:
#print('N Val Labels: {}'.format(len(set(val_labels))))

n_labels = len(set(train_labels))
print('N Train Labels: {}'.format(n_labels))

N Train Labels: 2


In [4]:
from transformers import TFAutoModel

def build_model(input_size, embedding_size, n_labels):
  bert = TFAutoModel.from_pretrained("bert-base-cased", output_hidden_states=False)

  input_ids = tf.keras.layers.Input(shape=(input_size,), name='input_ids', dtype='int32')
  input_token_types = tf.keras.layers.Input(shape=(input_size,), name='token_type_ids', dtype='int32')
  input_masks = tf.keras.layers.Input(shape=(input_size,), name='attention_mask', dtype='int32')

  x = bert.bert(input_ids, input_token_types, input_masks)[0]
  x = tf.keras.layers.GlobalMaxPool1D()(x)
  x = tf.keras.layers.Dense(embedding_size, activation='relu', name='embeddings')(x)
  x = tf.keras.layers.Dropout(0.2)(x)
  x = tf.keras.layers.Dense(n_labels)(x)

  clf_model = tf.keras.Model(
    inputs=[input_ids, input_token_types, input_masks],
    outputs = x
  )
  print(clf_model.summary())

  return clf_model

In [5]:
EPOCHS = 1
BATCH_SIZE = 8
LEARNING_RATE = 5e-5
TB_DIR = 'tensorboard'
EMBEDDING_SIZE = 256
input_size = 150
n_labels = 10

In [6]:
clf_model = build_model(input_size, EMBEDDING_SIZE, n_labels)

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 150)]        0                                            
__________________________________________________________________________________________________
token_type_ids (InputLayer)     [(None, 150)]        0                                            
__________________________________________________________________________________________________
attention_mask (InputLayer)     [(None, 150)]        0                                            
__________________________________________________________________________________________________
bert (TFBertMainLayer)          TFBaseModelOutputWit 10

In [12]:
clf_model.layers

In [8]:
for layer in clf_model.layers:
  if layer.name == 'bert':
    layer.trainable = False

print(clf_model.summary())

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 150)]        0                                            
__________________________________________________________________________________________________
token_type_ids (InputLayer)     [(None, 150)]        0                                            
__________________________________________________________________________________________________
attention_mask (InputLayer)     [(None, 150)]        0                                            
__________________________________________________________________________________________________
bert (TFBertMainLayer)          TFBaseModelOutputWit 108310272   input_ids[0][0]                  
                                                                 token_type_ids[0][0]         

In [182]:
clf_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[
        tf.metrics.SparseCategoricalAccuracy(),
        tf.keras.metrics.SparseTopKCategoricalAccuracy(k=3)
    ],
)

In [183]:
clf_model.fit(
  train_dataset.shuffle(500).batch(BATCH_SIZE),
  batch_size=BATCH_SIZE,
  epochs=EPOCHS,
  callbacks=[tf.keras.callbacks.TensorBoard(log_dir=TB_DIR)]
)

11/11 [==============================] - 232s 20s/step - loss: 0.7529 - sparse_categorical_accuracy: 0.8140 - sparse_top_k_categorical_accuracy: 1.0000


In [184]:
model = tf.keras.Model(
  inputs=clf_model.inputs,
  outputs = clf_model.get_layer('embeddings').output
)

In [30]:
with tarfile.open('model.tar.gz', "w:gz") as tar:
  tar.add('MTG_BERT/1', arcname=os.path.basename('MTG_BERT/1'))

FileNotFoundError: [WinError 2] The system cannot find the file specified: 'MTG_BERT/1'

In [29]:
model_path = 'model.tar.gz'
print('Extracting model from path: {}'.format(model_path))
with tarfile.open(model_path) as tar:
    tar.extractall(path='.')

Extracting model from path: model.tar.gz


In [41]:
train_texts[0]

'Vigilance\nYou may look at the top card of your library any time.\nYou may cast Aura and Equipment spells from the top of your library. When you cast an Equipment spell this way, it gains "When this Equipment enters the battlefield, attach it to target creature you control."'

In [42]:
train_encoding = tokenizer([train_texts[0]], padding='max_length', max_length=input_size, return_tensors="tf")
train_encoding

{'input_ids': <tf.Tensor: shape=(1, 150), dtype=int32, numpy=
array([[  101,   159, 24874, 13831,  1192,  1336,  1440,  1120,  1103,
         1499,  3621,  1104,  1240,  3340,  1251,  1159,   119,  1192,
         1336,  2641, 27758,  1611,  1105, 22897, 12168,  1121,  1103,
         1499,  1104,  1240,  3340,   119,  1332,  1128,  2641,  1126,
        22897,  5814,  1142,  1236,   117,  1122, 12535,   107,  1332,
         1142, 22897,  7603,  1103, 13777,   117, 25337,  1122,  1106,
         4010,  6093,  1128,  1654,   119,   107,   102,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0, 

In [88]:
cards_df = pd.read_csv('cardsS3.csv')\
  .reset_index(drop=True)\
  .fillna(value={'text': 'Blank'})

cards_txt = list(cards_df.text)
cards_name = [
  (name + '-' + str(id_val)).replace(' ','_').replace('//', 'II') for name, id_val in zip(cards_df.name, cards_df.id)
]

C:\Users\lwest\Anaconda3\envs\magicml\lib\site-packages\IPython\core\interactiveshell.py:3170: DtypeWarning: Columns (16,40) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [93]:
cards_df[['id','name','text']].query('name == "Galea, Kindler of Hope"')

,id,name,text
0,4751,"Galea, Kindler of Hope",Vigilance\nYou may look at the top card of you...
317,5068,"Galea, Kindler of Hope",Vigilance\nYou may look at the top card of you...
6249,38613,"Galea, Kindler of Hope",Vigilance\nYou may look at the top card of you...


In [96]:
cards_name[0]

'Galea,_Kindler_of_Hope-4751'

In [195]:
len(cards_txt)

11573

In [196]:
card_txt_tokens = tokenizer(cards_txt[0:50], padding='max_length', max_length=150, return_tensors="tf")

In [197]:
card_txt_dict = dict(card_txt_tokens)

In [186]:
embeddings = model(card_txt_tokens).numpy()
embeddings

array([[1.307891  , 0.        , 0.        , 0.14575239, 1.2158006 ,
        0.        , 0.        , 0.5810345 , 0.        , 0.16147217,
        0.        , 0.58082   , 0.7087351 , 0.32474077, 0.        ,
        0.        , 0.        , 0.03592885, 0.10091279, 0.5540062 ,
        0.9030094 , 0.08787068, 0.        , 1.4051552 , 1.017769  ,
        0.08638174, 0.24352022, 0.        , 0.        , 1.4627671 ,
        0.5749934 , 0.        , 0.        , 0.        , 0.45784137,
        0.        , 0.        , 0.5924064 , 0.        , 0.04069049,
        0.        , 0.        , 0.16152793, 0.        , 0.        ,
        0.1047825 , 0.        , 0.        , 0.26026517, 0.        ,
        0.        , 0.        , 0.42721033, 0.06578838, 0.        ,
        0.7935069 , 0.        , 0.07354768, 0.        , 0.        ,
        0.53210926, 0.60303867, 0.54622394, 0.01982088, 0.9150206 ,
        0.51029855, 0.8223913 , 0.4567781 , 0.        , 0.        ,
        0.        , 0.11725372, 0.52738553, 0.  

In [198]:
embeddings = model2.predict(card_txt_dict, batch_size=16)
embeddings

array([[1.3159626 , 0.36353502, 0.        , ..., 0.        , 0.        ,
        0.69775426],
       [1.6891388 , 0.19039054, 0.12856497, ..., 0.        , 0.        ,
        0.8787827 ],
       [1.6016773 , 0.28539756, 0.        , ..., 0.        , 0.1156975 ,
        0.82742465],
       ...,
       [1.614273  , 0.20177709, 0.        , ..., 0.        , 0.2009445 ,
        0.9007863 ],
       [1.9174874 , 0.27360064, 0.1795104 , ..., 0.        , 0.14034717,
        1.1323255 ],
       [1.9534429 , 0.08405267, 0.11816014, ..., 0.        , 0.07983588,
        1.0849926 ]], dtype=float32)

In [187]:
model.save('MTG_BERT/1')

C:\Users\lwest\Anaconda3\envs\magicml\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


INFO:tensorflow:Assets written to: MTG_BERT/1\assets


INFO:tensorflow:Assets written to: MTG_BERT/1\assets


In [188]:
#model2 = tf.saved_model.load('MTG_BERT/1')

In [191]:
model2 = tf.keras.models.load_model('MTG_BERT/1')

In [192]:
card_txt_tokens

{'input_ids': <tf.Tensor: shape=(1, 150), dtype=int32, numpy=
array([[  101,   159, 24874, 13831,  1192,  1336,  1440,  1120,  1103,
         1499,  3621,  1104,  1240,  3340,  1251,  1159,   119,  1192,
         1336,  2641, 27758,  1611,  1105, 22897, 12168,  1121,  1103,
         1499,  1104,  1240,  3340,   119,  1332,  1128,  2641,  1126,
        22897,  5814,  1142,  1236,   117,  1122, 12535,   107,  1332,
         1142, 22897,  7603,  1103, 13777,   117, 25337,  1122,  1106,
         4010,  6093,  1128,  1654,   119,   107,   102,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0, 

In [193]:
model2(card_txt_tokens)

<tf.Tensor: shape=(1, 256), dtype=float32, numpy=
array([[1.307891  , 0.        , 0.        , 0.14575239, 1.2158006 ,
        0.        , 0.        , 0.5810345 , 0.        , 0.16147217,
        0.        , 0.58082   , 0.7087351 , 0.32474077, 0.        ,
        0.        , 0.        , 0.03592885, 0.10091279, 0.5540062 ,
        0.9030094 , 0.08787068, 0.        , 1.4051552 , 1.017769  ,
        0.08638174, 0.24352022, 0.        , 0.        , 1.4627671 ,
        0.5749934 , 0.        , 0.        , 0.        , 0.45784137,
        0.        , 0.        , 0.5924064 , 0.        , 0.04069049,
        0.        , 0.        , 0.16152793, 0.        , 0.        ,
        0.1047825 , 0.        , 0.        , 0.26026517, 0.        ,
        0.        , 0.        , 0.42721033, 0.06578838, 0.        ,
        0.7935069 , 0.        , 0.07354768, 0.        , 0.        ,
        0.53210926, 0.60303867, 0.54622394, 0.01982088, 0.9150206 ,
        0.51029855, 0.8223913 , 0.4567781 , 0.        , 0.        

In [81]:
corr = np.inner(embeddings, embeddings)

In [87]:
'Galea,_Kindler_of_Hope-4751' in cards_name[0:1000]

True

In [100]:
pd.DataFrame(corr, columns=cards_name[0:1000], index=cards_name[0:1000])\
  [['Galea,_Kindler_of_Hope-4751']]\
  .sort_values(by='Galea,_Kindler_of_Hope-4751', ascending=False)

,"Galea,_Kindler_of_Hope-4751"
Oketra_the_True-5832,189.835449
Grand_Master_of_Flowers-5099,182.591049
Grand_Master_of_Flowers-5364,182.590988
Aven_Wind_Guide-6038,177.246506
Majestic_Myriarch-6008,176.364075
...,...
Steadfast_Paladin-5120,101.102097
Ramunap_Excavator-6020,100.779015
Pouncing_Cheetah-6016,100.012436
Cut_II_Ribbons-6043,99.611000


In [206]:
res = clf_model(train_encoding)
print(res)
print(tf.nn.softmax(res, axis=-1))
np.argmax(tf.nn.softmax(res, axis=-1).numpy())

tf.Tensor(
[[-1.4388261  -0.79059803 -1.6125364  -0.3912301   1.1768957  -0.61361474
   0.40323156  0.593866    0.736947    1.2922086 ]], shape=(1, 10), dtype=float32)
tf.Tensor(
[[0.01648402 0.03151992 0.01385549 0.04699248 0.2254534  0.03762253
  0.10400607 0.12584913 0.14520766 0.25300932]], shape=(1, 10), dtype=float32)


9

In [209]:
model = tf.keras.Model(
  inputs=clf_model.inputs,
  outputs = clf_model.get_layer('embeddings').output
)

In [213]:
res = model(train_encoding)
res.numpy()

array([[0.1623031 , 0.        , 0.7343738 , 0.40971297, 0.35925984,
        0.        , 0.        , 0.        , 0.        , 0.06207182,
        0.        , 0.        , 0.06542838, 0.19620104, 0.        ,
        0.4549396 , 0.11906414, 0.35826373, 0.        , 0.32381967,
        0.        , 0.        , 0.        , 0.        , 0.78335834,
        0.4621384 , 0.        , 0.        , 0.        , 0.        ,
        0.7057188 , 0.        , 0.        , 0.12593283, 0.        ,
        0.        , 1.0365506 , 0.03457262, 0.18150075, 0.        ,
        0.08478717, 0.        , 0.        , 0.43191472, 0.        ,
        0.        , 0.        , 0.8382378 , 0.0845039 , 0.71750885,
        0.59217596, 0.        , 0.        , 0.        , 0.        ,
        0.7069869 , 0.        , 0.        , 0.        , 0.90636104,
        0.        , 0.        , 0.9120938 , 0.22369602, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.15908827,
        0.        , 0.        , 0.        , 0.  

***

In [1]:
import pandas as pd

In [4]:
supported_sets = [
  'Forgotten Realms',
  'Strixhaven',
  'Throne of Eldraine',
  'Theros Beyond Death',
  'Ikoria: Lair of Behemoths',
  'Core Set 2021',
  'Zendikar Rising',
  'Kaldheim',
  'Core Set 2020',
  'War of the Spark',
  'Ravnica Allegiance',
  'Guilds of Ravnica',
  'Ixalan',
  'Rivals of Ixalan',
  'Dominaria',
  'Core Set 2019',
  'Historic Anthology',
  'Historic Anthology 2',
  'Historic Anthology 3',
  'Jumpstart',
  'Amonkhet Remastered',
  'Kaladesh Remastered'
]

In [5]:
LOCAL_RAW_PATH = '../data/mtgjson'

In [7]:
def get_image_uris(row):
  try:
      if pd.notna(row['image_uris']):
          return row['image_uris']
      else:
          return [card['image_uris'] for card in row['card_faces']]
  except:
      return 'Blank'

In [8]:
# Prep data
# Get MTGJSON data
cards_df = pd.read_csv(LOCAL_RAW_PATH + '/cards.csv')\
  .drop(columns=['index'])

# Merge with sets data
sets_df = pd.read_csv(LOCAL_RAW_PATH + '/sets.csv')[['code','name']]\
  .rename(columns={'name': 'setName', 'code':'setCode'})

cards_df = cards_df\
  .merge(sets_df, how='left', on='setCode')

# Merge with legalities / formats data
legs_df = pd.read_csv(LOCAL_RAW_PATH + '/legalities.csv')\
  .drop_duplicates(subset=['uuid','format','status'])\
  .pivot(index='uuid', columns='format', values='status')\
  .reset_index()\
  .fillna('Blank')

cards_df = cards_df\
  .merge(legs_df, how='left', on='uuid')

# Merge with scryfall data
scryfall_df = pd.read_json(LOCAL_RAW_PATH + '/scryfall_cards.json')
scryfall_sets = scryfall_df.set_name.unique()
supported_sets_varients = [scry_s for scry_s in scryfall_sets if any([s in scry_s for s in supported_sets])]

scryfall_df = scryfall_df\
    .query('set_name == @supported_sets_varients')\
    [['id','image_uris','card_faces']]\
    .reset_index(drop=True)\
    .assign(image_urls=lambda df: df.apply(get_image_uris, axis=1))\
    .drop(columns=['image_uris','card_faces'])\
    .rename(columns={'id': 'scryfallId'})

cards_df = cards_df\
  .merge(scryfall_df, how='left', on='scryfallId')

C:\Users\lwest\Anaconda3\envs\magicml\lib\site-packages\IPython\core\interactiveshell.py:3170: DtypeWarning: Columns (3,12,16,19,40,51) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [10]:
print(cards_df.shape)
cards_df.head(5)

(61998, 93)


,id,artist,asciiName,availability,borderColor,cardKingdomFoilId,cardKingdomId,colorIdentity,colorIndicator,colors,...,modern,oldschool,pauper,paupercommander,penny,pioneer,premodern,standard,vintage,image_urls
0,1,Pete Venters,NaN,"mtgo,paper",black,NaN,122719.0,W,NaN,W,...,Legal,Blank,Blank,Restricted,Legal,Blank,Legal,Blank,Legal,NaN
1,2,Volkan Baǵa,NaN,"mtgo,paper",black,NaN,122720.0,W,NaN,W,...,Legal,Blank,Legal,Legal,Legal,Blank,Legal,Blank,Legal,NaN
2,3,Justin Sweet,NaN,"mtgo,paper",black,NaN,122725.0,W,NaN,W,...,Legal,Blank,Legal,Legal,Legal,Blank,Legal,Blank,Legal,NaN
3,4,Matthew D. Wilson,NaN,"mtgo,paper",black,123094.0,122726.0,W,NaN,W,...,Legal,Blank,Blank,Restricted,Legal,Blank,Legal,Blank,Legal,NaN
4,5,Rebecca Guay,NaN,"mtgo,paper",black,123095.0,122727.0,W,NaN,W,...,Legal,Blank,Legal,Legal,Legal,Blank,Legal,Blank,Legal,NaN


In [1]:
import pandas as pd

In [2]:
embed_df = pd.read_parquet('../data/cards_embeddings.parquet')
print(embed_df.shape)
embed_df.head(5)

(10373, 10373)


,"Galea,_Kindler_of_Hope-4751","Prosper,_Tome-Bound-4752",Sefris_of_the_Hidden_Ways-4753,"Vrondiss,_Rage_of_Ancients-4754",Fey_Steed-4755,Holy_Avenger-4756,Immovable_Rod-4757,Mantle_of_the_Ancients-4758,Radiant_Solar-4759,Revivify-4760,...,Swamp-61989,Mountain-61990,Forest-61991,"Orah,_Skyclave_Hierophant-61992","Charix,_the_Raging_Isle-61993",Into_the_Roil-61994,Bloodchief's_Thirst-61995,Roil_Eruption-61996,Roiling_Regrowth-61997,Kargan_Warleader-61998
"Galea,_Kindler_of_Hope-4751",1.000000,0.561341,0.629551,0.576591,0.672578,0.635352,0.513502,0.731142,0.529273,0.577024,...,0.015822,0.000856,0.016533,0.575165,0.555767,0.592809,0.572976,0.479776,0.533010,0.380926
"Prosper,_Tome-Bound-4752",0.561341,1.000000,0.564873,0.521453,0.551346,0.427444,0.463979,0.580434,0.561041,0.501172,...,0.068526,0.033384,0.044551,0.541683,0.533398,0.528526,0.502451,0.398537,0.537174,0.296284
Sefris_of_the_Hidden_Ways-4753,0.629551,0.564873,1.000000,0.554480,0.638599,0.494233,0.455184,0.680465,0.611876,0.645469,...,0.020438,0.030995,0.029236,0.558692,0.478554,0.500377,0.465825,0.414055,0.493432,0.361058
"Vrondiss,_Rage_of_Ancients-4754",0.576591,0.521453,0.554480,1.000000,0.582828,0.546531,0.414580,0.694726,0.537554,0.612475,...,0.072991,0.110146,0.106410,0.525995,0.482405,0.549149,0.536154,0.547571,0.508484,0.440132
Fey_Steed-4755,0.672578,0.551346,0.638599,0.582828,1.000000,0.586323,0.561870,0.679132,0.533896,0.493466,...,0.005874,-0.020226,-0.015497,0.592051,0.575023,0.602637,0.605733,0.468395,0.465949,0.516642


In [11]:
card = 'Audacious_Thief-AJMP'

In [12]:
[c for c in embed_df.columns if 'Audacious_Thief' in c]

['Audacious_Thief-5491', 'Audacious_Thief-31377']

In [10]:
embed_df.reset_index()\
  .rename(columns={'index': 'Names'})\
  [['Names', card]]

KeyError: "['Audacious_Thief-AJMP'] not in index"

In [12]:
LOCAL_CLEAN_PATH = '../'

In [13]:
# Save cards for NLP in local EFS
cards_df\
  .query('setName == @supported_sets_varients')\
  .to_csv(LOCAL_CLEAN_PATH + '/cards.csv', index=False)

# Save ALL cards in local EFS
cards_df\
  .to_csv(LOCAL_CLEAN_PATH + '/all_cards.csv', index=False)

In [30]:
# Test Free Text Query
query = 'extra_turn'
embed_query = use_embed([query])
print(embed_query.shape)

(1, 512)


In [13]:
sim = np.inner(all_embeds, embed_query)
print(sim.shape)

(7998, 1)


In [31]:
sims_list = pd.DataFrame(sim, columns=['free_text_query'], index=cards_name)\
    .sort_values(by='free_text_query', ascending=False)\
    .head(50)\
    .reset_index()\
    .rename(columns={'index':'Names', 'free_text_query':'similarity'})\
    .merge(cards_df, how='left', on='Names')\
    .assign(similarity=lambda df: df.similarity.astype('str'))\
    .assign(id=lambda df: df.id.astype('str'))\
    .assign(mtgArenaId=lambda df: df.mtgArenaId.astype('str'))\
    .assign(loyalty=lambda df: df.loyalty.astype('str'))\
    .assign(power=lambda df: df.power.astype('str'))\
    .assign(toughness=lambda df: df.toughness.astype('str'))\
    .assign(convertedManaCost=lambda df: df.convertedManaCost.astype('str'))\
    .to_dict(orient='records')

sims_list[0]

{'Names': 'Time_Warp-17722',
 'similarity': '0.44100022',
 'id': '17722',
 'mtgArenaId': '0',
 'scryfallId': '87fe8637-9be1-4b57-832f-38cf92d23482',
 'name': 'Time Warp',
 'colors': 'U',
 'setName': 'Explorers of Ixalan',
 'convertedManaCost': '5.0',
 'manaCost': '{3}{U}{U}',
 'loyalty': '0',
 'power': '0',
 'toughness': '0',
 'type': 'Sorcery',
 'types': 'Sorcery',
 'subtypes': '0',
 'text': 'Target player takes an extra turn after this one.',
 'image_urls': "{'small': 'https://c1.scryfall.com/file/scryfall-cards/small/front/8/7/87fe8637-9be1-4b57-832f-38cf92d23482.jpg?1562927104', 'normal': 'https://c1.scryfall.com/file/scryfall-cards/normal/front/8/7/87fe8637-9be1-4b57-832f-38cf92d23482.jpg?1562927104', 'large': 'https://c1.scryfall.com/file/scryfall-cards/large/front/8/7/87fe8637-9be1-4b57-832f-38cf92d23482.jpg?1562927104', 'png': 'https://c1.scryfall.com/file/scryfall-cards/png/front/8/7/87fe8637-9be1-4b57-832f-38cf92d23482.png?1562927104', 'art_crop': 'https://c1.scryfall.com/fil

In [32]:
Item = {
    'freeText': query,
    'similarities': sims_list
}

In [11]:
merge_cols = [
    'Names','id','mtgArenaId','scryfallId','name','colors','setName',
    'convertedManaCost','manaCost','loyalty','power','toughness',
    'type','types','subtypes','text','image_urls',
    'brawl','commander','duel','future','historic','legacy','modern',
    'oldschool','pauper','penny','pioneer','standard','vintage'
]

cards_df = pd.read_csv('cards.csv')\
    .query('mtgArenaId.notnull()')\
    .assign(Names=lambda df: df.name + '-' + df.id.astype('str'))\
    .assign(Names=lambda df: df.Names.apply(lambda x: x.replace(' ', '_').replace('//', 'II')))\
    .fillna('0')\
    [merge_cols]

print(cards_df.shape)
cards_df.head(2)

(5419, 30)


,Names,id,mtgArenaId,scryfallId,name,colors,setName,convertedManaCost,manaCost,loyalty,...,future,historic,legacy,modern,oldschool,pauper,penny,pioneer,standard,vintage
4753,Archon_of_Sun's_Grace-4754,4754,74983.0,94f05268-0d4f-4638-aec3-a85fc339e3a7,Archon of Sun's Grace,W,Jumpstart Arena Exclusives,4.0,{2}{W}{W},0,...,Legal,Legal,Legal,Legal,Blank,Blank,Blank,Legal,Legal,Legal
4754,Audacious_Thief-4755,4755,74991.0,ba315deb-d5a9-4013-b6ef-e4efe652e569,Audacious Thief,B,Jumpstart Arena Exclusives,3.0,{2}{B},0,...,Blank,Legal,Legal,Legal,Blank,Legal,Blank,Legal,Blank,Legal


In [29]:
embed_df.columns.nunique()

5420

In [30]:
cards_df.query('Names == "Zof_Consumption_II_Zof_Bloodbog-55942"')

,Names,id,mtgArenaId,scryfallId,name,colors,setName,convertedManaCost,manaCost,loyalty,...,future,historic,legacy,modern,oldschool,pauper,penny,pioneer,standard,vintage
55941,Zof_Consumption_II_Zof_Bloodbog-55942,55942,73331.0,98496d5b-1519-4f0c-8b46-0a43be643dfb,Zof Consumption // Zof Bloodbog,B,Zendikar Rising,6.0,{4}{B}{B},0,...,Legal,Legal,Legal,Legal,Blank,Blank,Legal,Legal,Legal,Legal


In [362]:
all_cards = embed_df.columns

BATCH_SIZE = 100
batches = [list(all_cards[n:n+BATCH_SIZE]) for n in range(1, len(all_cards), BATCH_SIZE)]

In [21]:
merge_cols = [
    'Names','id','mtgArenaId','scryfallId','name','colors','setName',
    'convertedManaCost','manaCost','loyalty','power','toughness',
    'type','types','subtypes','text',
    'brawl','commander','duel','future','historic','legacy','modern',
    'oldschool','pauper','penny','pioneer','standard','vintage'
]

In [22]:
cards_df = pd.read_csv('cards.csv')\
    .query('mtgArenaId.notnull()')\
    .assign(Names=lambda df: df.name + '-' + df.id.astype('str'))\
    .assign(Names=lambda df: df.Names.apply(lambda x: x.replace(' ', '_').replace('//', 'II')))\
    .fillna('0')\
    [merge_cols]

print(cards_df.shape)
cards_df.head(3)

(5419, 29)


,Names,id,mtgArenaId,scryfallId,name,colors,setName,convertedManaCost,manaCost,loyalty,...,future,historic,legacy,modern,oldschool,pauper,penny,pioneer,standard,vintage
4753,Archon_of_Sun's_Grace-4754,4754,74983.0,94f05268-0d4f-4638-aec3-a85fc339e3a7,Archon of Sun's Grace,W,Jumpstart Arena Exclusives,4.0,{2}{W}{W},0,...,Legal,Legal,Legal,Legal,Blank,Blank,Blank,Legal,Legal,Legal
4754,Audacious_Thief-4755,4755,74991.0,ba315deb-d5a9-4013-b6ef-e4efe652e569,Audacious Thief,B,Jumpstart Arena Exclusives,3.0,{2}{B},0,...,Blank,Legal,Legal,Legal,Blank,Legal,Blank,Legal,Blank,Legal
4755,Banishing_Light-4756,4756,74986.0,ca112bae-6ac5-4cdf-9e8c-1b99f7396995,Banishing Light,W,Jumpstart Arena Exclusives,3.0,{2}{W},0,...,Legal,Legal,Legal,Legal,Blank,Blank,Legal,Legal,Legal,Legal


In [23]:
cards_df.Names.nunique()

5419

In [24]:
cards_df.name.nunique()

4267

In [25]:
cards_df.scryfallId.nunique()

5283

In [38]:
card = 'Oreskos_Swiftclaw-28847'

In [44]:
staged_card = embed_df[['Names', card]]\
    .merge(cards_df, how='left', on='Names')

Item = staged_card.query('Names == @card').to_dict(orient='records')

staged_card = staged_card\
    .sort_values(by=card, ascending=False)\
    .head(51)\
    .rename(columns={card: 'similarity'})\
    .assign(similarity=lambda df: df.similarity.astype('str'))\
    .assign(id=lambda df: df.id.astype('str'))\
    .assign(mtgArenaId=lambda df: df.mtgArenaId.astype('str'))\
    .assign(loyalty=lambda df: df.loyalty.astype('str'))\
    .assign(power=lambda df: df.power.astype('str'))\
    .assign(toughness=lambda df: df.toughness.astype('str'))\
    .assign(convertedManaCost=lambda df: df.convertedManaCost.astype('str'))

staged_card

,Names,similarity,id,mtgArenaId,scryfallId,name,colors,setName,convertedManaCost,manaCost,...,future,historic,legacy,modern,oldschool,pauper,penny,pioneer,standard,vintage
3884,Axebane_Beast-45872,0.9999998999999999,45872,69249.0,2f420b35-1f73-41c8-a15f-1aee4af0999c,Axebane Beast,G,Ravnica Allegiance,4.0,{3}{G},...,Blank,Legal,Legal,Legal,Blank,Legal,Legal,Legal,Blank,Legal
3684,Canal_Monitor-45672,0.9999998999999999,45672,66743.0,78226edc-87dd-4c38-987c-52aefe0f9531,Canal Monitor,B,Rivals of Ixalan,5.0,{4}{B},...,Blank,Legal,Legal,Legal,Blank,Legal,Legal,Legal,Blank,Legal
2901,Tolarian_Scholar-28942,0.9999998999999999,28942,67840.0,2eda67da-02b5-4ecb-9038-10e026d454ec,Tolarian Scholar,U,Core Set 2019,3.0,{2}{U},...,Blank,Legal,Legal,Legal,Blank,Legal,Legal,Legal,Blank,Legal
4864,Headwater_Sentries-54863,0.9999998999999999,54863,66077.0,2af2c338-f5e9-4596-9435-c6aa965ae541,Headwater Sentries,U,Ixalan,4.0,{3}{U},...,Blank,Legal,Legal,Legal,Blank,Legal,Legal,Legal,Blank,Legal
427,Armored_Whirl_Turtle-5931,0.9999998999999999,5931,75465.0,44a783f2-04d3-42fc-acc1-5f2974f9aca2,Armored Whirl Turtle,U,Arena Beginner Set,3.0,{2}{U},...,Blank,Legal,Legal,Blank,Blank,Legal,Blank,Blank,Blank,Legal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3598,Treetop_Warden-35145,0.9999998999999999,35145,69119.0,771341f5-11b2-4edc-aa41-088e852c058e,Treetop Warden,G,Arena New Player Experience Cards,2.0,{1}{G},...,Blank,Legal,Blank,Blank,Blank,Blank,Blank,Blank,Blank,Blank
4749,Rumbling_Baloth-54748,0.9999998999999999,54748,68801.0,a366fd5c-f6cd-4773-a25d-6114e11238cc,Rumbling Baloth,G,Arena New Player Experience Extras,4.0,{2}{G}{G},...,Blank,Legal,Legal,Legal,Blank,Legal,Legal,Legal,Blank,Legal
3053,Goblin_Assailant-29095,0.9999998999999999,29095,70114.0,8cfefb65-b6e4-44a1-baa9-d3c00ee8ba96,Goblin Assailant,R,Core Set 2020,2.0,{1}{R},...,Blank,Legal,Legal,Legal,Blank,Legal,Legal,Legal,Blank,Legal
786,Tolarian_Scholar-16832,0.9999998999999999,16832,67246.0,00d89839-60d7-4de2-a78a-1afdcc21c053,Tolarian Scholar,U,Dominaria,3.0,{2}{U},...,Blank,Legal,Legal,Legal,Blank,Legal,Legal,Legal,Blank,Legal


In [40]:
Item = staged_card.query('Names == @card').to_dict(orient='records')
Item

[{'Names': 'Oreskos_Swiftclaw-28847',
  'similarity': '0.9999998999999999',
  'id': '28847',
  'mtgArenaId': '67742.0',
  'scryfallId': '0ea1dfb4-1983-41f7-956c-f2a1d1489b54',
  'name': 'Oreskos Swiftclaw',
  'colors': 'W',
  'setName': 'Core Set 2019',
  'convertedManaCost': '2.0',
  'manaCost': '{1}{W}',
  'loyalty': '0',
  'power': '3',
  'toughness': '1',
  'type': 'Creature — Cat Warrior',
  'types': 'Creature',
  'subtypes': 'Cat,Warrior',
  'text': '0',
  'brawl': 'Blank',
  'commander': 'Legal',
  'duel': 'Legal',
  'future': 'Blank',
  'historic': 'Legal',
  'legacy': 'Legal',
  'modern': 'Legal',
  'oldschool': 'Blank',
  'pauper': 'Legal',
  'penny': 'Legal',
  'pioneer': 'Legal',
  'standard': 'Blank',
  'vintage': 'Legal'}]

## Get card item from Dyanmo

In [1]:
import os
import json
import boto3
from boto3.dynamodb.conditions import Key

In [2]:
SIMILARITY_TABLE = 'similarity-dev'

In [3]:
boto_sess = boto3.Session(profile_name='lw2134')
dynamodb = boto_sess.resource('dynamodb')
table = dynamodb.Table(SIMILARITY_TABLE)

In [8]:
# QUERY CARD BY NAME CONTAINS
card_item = table.query(
    KeyConditions={
        'name': {
            'AttributeValueList': ['Banishing'],
            'ComparisonOperator': 'BEGINS_WITH'
        }
    }
)

print(card_item)
#print(card_item['Items'][0]['name'])
#print(len(card_item['Items']))
#print(card_item['Items'][0]['text'])

ClientError: An error occurred (ValidationException) when calling the Query operation: Query key condition not supported

In [323]:
# QUERY BY NAME
card_item = table.query(
    KeyConditionExpression=Key('name').eq("Bontu's Last Reckoning")
)

print(card_item['Items'][0]['name'])
print(len(card_item['Items']))
print(card_item['Items'][0]['text'])

Bontu's Last Reckoning
1
Destroy all creatures. Lands you control don't untap during your next untap step.


## Call deployed API

In [10]:
import requests

### Scryfall Card Search API

In [23]:
q = 'garruk cursed'

In [24]:
res = requests.get('https://api.scryfall.com/cards/search?q={}'.format(q))

In [25]:
res.json()

{'object': 'list',
 'total_cards': 2,
 'has_more': False,
 'data': [{'object': 'card',
   'id': 'abef512f-8f1d-4257-b16f-c0eed58670ec',
   'oracle_id': 'e0cef79c-ad47-4cbc-9d73-a913e487ccb7',
   'multiverse_ids': [473153],
   'mtgo_id': 78526,
   'arena_id': 70338,
   'tcgplayer_id': 198500,
   'cardmarket_id': 398939,
   'name': 'Garruk, Cursed Huntsman',
   'lang': 'en',
   'released_at': '2019-10-04',
   'uri': 'https://api.scryfall.com/cards/abef512f-8f1d-4257-b16f-c0eed58670ec',
   'scryfall_uri': 'https://scryfall.com/card/eld/191/garruk-cursed-huntsman?utm_source=api',
   'layout': 'normal',
   'highres_image': True,
   'image_uris': {'small': 'https://c1.scryfall.com/file/scryfall-cards/small/front/a/b/abef512f-8f1d-4257-b16f-c0eed58670ec.jpg?1572490758',
    'normal': 'https://c1.scryfall.com/file/scryfall-cards/normal/front/a/b/abef512f-8f1d-4257-b16f-c0eed58670ec.jpg?1572490758',
    'large': 'https://c1.scryfall.com/file/scryfall-cards/large/front/a/b/abef512f-8f1d-4257-b16

### MagicML Semantic Search API

In [390]:
query = {
  "key": "name",
  "value": "Murder"
}

In [393]:
res = requests.post('https://38axjswipg.execute-api.us-east-1.amazonaws.com/dev/query', json=query)

if len(res.json()['cards']) > 0:
    print(res.json()['cards'][0]['Names'])
    print(res.json()['cards'][0]['name'])
    print(len(res.json()['cards']))
else:
    print(res.json()['cards'])

Murder-29173
Murder
1


In [394]:
res.json()

{'cards': [{'subtypes': '0',
   'setName': 'Core Set 2020',
   'duel': 'Legal',
   'mtgArenaId': '69894.0',
   'scryfallId': '6a2b22bc-e81b-4f27-a52b-9f3edad25439',
   'types': 'Instant',
   'brawl': 'Blank',
   'name': 'Murder',
   'penny': 'Legal',
   'pioneer': 'Legal',
   'convertedManaCost': '3.0',
   'modern': 'Legal',
   'colors': 'B',
   'id': '29173',
   'future': 'Blank',
   'historic': 'Legal',
   'commander': 'Legal',
   'toughness': '0',
   'loyalty': '0',
   'pauper': 'Legal',
   'Names': 'Murder-29173',
   'text': 'Destroy target creature.',
   'image_urls': "{'small': 'https://c1.scryfall.com/file/scryfall-cards/small/front/6/a/6a2b22bc-e81b-4f27-a52b-9f3edad25439.jpg?1592516737', 'normal': 'https://c1.scryfall.com/file/scryfall-cards/normal/front/6/a/6a2b22bc-e81b-4f27-a52b-9f3edad25439.jpg?1592516737', 'large': 'https://c1.scryfall.com/file/scryfall-cards/large/front/6/a/6a2b22bc-e81b-4f27-a52b-9f3edad25439.jpg?1592516737', 'png': 'https://c1.scryfall.com/file/scryfal

## Lambda Cards Sorter Master

In [39]:
STAGE = 'dev'

In [40]:
_, s3, boto_sess = aws_connect('s3', 'lw2134', session=True)
lambda_client = boto_sess.client('lambda')

In [41]:
all_cards = pred_df.columns

In [47]:
BATCHES_OF = 10
batches = [list(all_cards[n:n+BATCHES_OF]) for n in range(1, len(all_cards), BATCHES_OF)]
print(len(batches))
batches[0]

542


["Archon_of_Sun's_Grace-AJMP",
 'Audacious_Thief-AJMP',
 'Banishing_Light-AJMP',
 'Bond_of_Revival-AJMP',
 'Carnifex_Demon-AJMP',
 'Doomed_Necromancer-AJMP',
 'Dryad_Greenseeker-AJMP',
 'Fanatic_of_Mogis-AJMP',
 'Gadwick,_the_Wizened-AJMP',
 'Goblin_Oriflamme-AJMP']

In [49]:
import json

In [54]:
json.dumps(batches[0][0:5])

'["Archon_of_Sun\'s_Grace-AJMP", "Audacious_Thief-AJMP", "Banishing_Light-AJMP", "Bond_of_Revival-AJMP", "Carnifex_Demon-AJMP"]'

In [141]:
for cards in batches:
    payload = {'cards': cards}

    res = lambda_client.invoke(
        FunctionName='magicml-similarity-{}-stage_embed_worker'.format(STAGE),
        InvocationType='Event',
        Payload=json.dumps(payload)
    )

In [142]:
a_card = pd.read_csv('sorted/Banishing_Light-AJMP.csv')
print(a_card.shape)

(7639, 33)


In [149]:
a_card.head(3)

,Banishing_Light-AJMP,id,mtgArenaId,scryfallId,name,colorIdentity,colors,setName,convertedManaCost,manaCost,...,future,historic,legacy,modern,oldschool,pauper,penny,pioneer,standard,vintage
0,1.000000,4756,74986.0,ca112bae-6ac5-4cdf-9e8c-1b99f7396995,Banishing Light,W,W,Jumpstart Arena Exclusives,3.0,{2}{W},...,Legal,Legal,Legal,Legal,Blank,Blank,Legal,Legal,Legal,Legal
1,1.000000,49136,70515.0,a1ddd113-140f-49c9-b45c-cf1b0d1dffd8,Banishing Light,W,W,Theros Beyond Death,3.0,{2}{W},...,Legal,Legal,Legal,Legal,Blank,Blank,Legal,Legal,Legal,Legal
2,0.773027,28775,67708.0,197743cd-249c-42ba-ac8d-027c088f8418,Hieromancer's Cage,W,W,Core Set 2019,4.0,{3}{W},...,Blank,Legal,Legal,Legal,Blank,Blank,Blank,Legal,Blank,Legal
